In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
import re
from time import sleep
import pandas as pd
from urllib.request import urlopen

In [3]:
df = pd.read_csv('data/petition_crawling_low_birth_rate.csv', sep=',', encoding='UTF-8')
len(df)

3339

In [4]:
df.head()

,Unnamed: 0,제목,청원시작,청원인원,Link
0,0,저출산 관련 육아휴직 및 어린이집 증설,2018-10-31,3,https://www1.president.go.kr/petitions/427334
1,1,저출산 ↔ 어린이집 부족. 서로 너무 상충되는 관계 아닌가요?,2018-10-31,7,https://www1.president.go.kr/petitions/427316
2,2,어린이집을 유지해주세요,2018-10-31,476,https://www1.president.go.kr/petitions/426902
3,3,오늘 안하면 늦습니다. 누구나 마음놓고 출산할 수 있는 제도 마련 시급합니다.,2018-10-31,2,https://www1.president.go.kr/petitions/426884
4,4,가족돌봄휴가제도를 조속히 시행해야 합니다.,2018-10-31,2,https://www1.president.go.kr/petitions/426830


In [5]:
df.tail(10)

,Unnamed: 0,제목,청원시작,청원인원,Link
3329,3329,인터넷실명제가 꼭 필요한 시점입니다.,2018-01-02,1,https://www1.president.go.kr/petitions/78631
3330,3330,어린이집 cctv를폐쇄하여 주시기를 바랍니다,2018-01-02,77,https://www1.president.go.kr/petitions/78611
3331,3331,아동수당을 주세요,2018-01-02,4,https://www1.president.go.kr/petitions/78479
3332,3332,미래 세대를 위한 펀드 조성을 위한 제안,2018-01-02,1,https://www1.president.go.kr/petitions/78412
3333,3333,동성결혼 또는 시민결합제도의 합법화와 '성소수자'에 대한 전체적인 사회교육 실행.,2018-01-02,229,https://www1.president.go.kr/petitions/78358
3334,3334,인구절벽 극복하기 정책제안,2018-01-02,1,https://www1.president.go.kr/petitions/78340
3335,3335,직장맘이라 죄인입니다.,2018-01-02,1,https://www1.president.go.kr/petitions/78284
3336,3336,태어난 아이라도 지켜주세요,2018-01-01,1,https://www1.president.go.kr/petitions/77939
3337,3337,국민 모두가 웃을 수 있는 나라를 만들어 주세요,2018-01-01,0,https://www1.president.go.kr/petitions/77852
3338,3338,원영이사건 고준희사건 인삼밭9개월유기사망사건 금천구신생아유기사망사건 광주5살3살15...,2018-01-01,6,https://www1.president.go.kr/petitions/77810


In [6]:
url = df.Link[5]
html = urlopen(url)
soup = BeautifulSoup(html, 'lxml')
soup.title

<title>우리 아이들을 위해서..... 사립유치원 해결 방안 제안합니다. &gt; 국민청원 &gt; 대한민국 청와대</title>

In [7]:
body = soup.find('div', 'View_write').get_text()
body[:500]

'\r\n\t\t\t\t\t\t\t\t\t\t\t교육의 바탕이 되어야할 사립유치원의 행태에 분노를 느낍니다. \r\n\t\t\t\t\t\t\t\t\t\t\t사교육의 병폐는 오늘 어제의 이야기는 아니지만 현재 보여주고 있는 사립유치원의 집단 행동과 \r\n\t\t\t\t\t\t\t\t\t\t\t이를 이용한 부의 축적은 진정한 교육과는 거리가 있는듯 합니다. \r\n\t\t\t\t\t\t\t\t\t\t\t아이를 키우며 고등교육과 맞먹는 유치원 비용에 놀라움을 느끼기도 했었기에... \n\r\n\t\t\t\t\t\t\t\t\t\t\t국가는 소중한 우리 아이들의 미래를 위하여 금번기회를 이용해 유아교육의 재정비를 촉구하는 바입니다. \n\r\n\t\t\t\t\t\t\t\t\t\t\t사립유치원의 개선을 위하여 유치교육의 대하여 제안합니다. \n\r\n\t\t\t\t\t\t\t\t\t\t\t 현 8세입학의 초등학교 입학과 별개로 5~7세 프리스쿨제를 제안합니다. \n\r\n\t\t\t\t\t\t\t\t\t\t\t미취학 아동의 취학전 전국 초등학교를 이용하여 입학전 기초교육 및 감성교육등 미취학생교육을 각 초등학교에서 \r\n\t\t\t\t\t\t\t\t\t\t\t시행하게 된다면, 기존 저출산으로 인한 초등학'

In [8]:
body = body.replace('\r', '').replace('\n', '').replace('\t', '')

In [9]:
# 본문 크롤링하기
add_body = []
count = 0

for link in df.Link[547:]:
    url = link
    html = urlopen(url)
    soup = BeautifulSoup(html, 'lxml')
    print(soup.title)
    
    body = soup.find('div', 'View_write').get_text()
    body = body.replace('\r', '').replace('\n', '').replace('\t', '')
    print('body:', body[:50])
    add_body.append(body)
    
    count += 1
    sleep(1)
    if count % 10 == 0:
        print('count:', count) 

<title>저출산 관련 육아휴직 및 어린이집 증설 &gt; 국민청원 &gt; 대한민국 청와대</title>
body: 300인 미만 기업에 다니고 있는 2018년도에 출산한 아기 엄마입니다. 먼저 회사에서는 
<title>저출산 ↔ 어린이집 부족. 서로 너무 상충되는 관계 아닌가요? &gt; 국민청원 &gt; 대한민국 청와대</title>
body: 안녕하세요. 천안시 백석동에 사는 한 시민입니다. 천안은 (특히 백석, 불당) 전국 통계상
<title>어린이집을 유지해주세요 &gt; 국민청원 &gt; 대한민국 청와대</title>
body: 양천구 신월동에 위치한 *** 국공립 어린이집에 6살 아이를 보내는 엄마입니다. 4년을 보
<title>오늘 안하면 늦습니다.  누구나 마음놓고 출산할 수 있는 제도 마련 시급합니다. &gt; 국민청원 &gt; 대한민국 청와대</title>
body: 안녕하세요 문재인대통령님^^ 국민의 한사람으로서 또한 애를 키우고 있는 가장으로서 대통령님
<title>가족돌봄휴가제도를 조속히 시행해야 합니다. &gt; 국민청원 &gt; 대한민국 청와대</title>
body: 올해 2월에 신문들에 일제히 보도된 가족돌봄휴가제도에 대한 내용입니다. [한겨레 2018.
<title>우리 아이들을 위해서..... 사립유치원 해결 방안 제안합니다. &gt; 국민청원 &gt; 대한민국 청와대</title>
body: 교육의 바탕이 되어야할 사립유치원의 행태에 분노를 느낍니다. 사교육의 병폐는 오늘 어제의 
<title>양육비 안주는 나쁜 아빠들 &gt; 국민청원 &gt; 대한민국 청와대</title>
body: 저는 두아이를 키우며 살고 있는 엄마입니다.  저는 10년 전  재판이혼을 하고 친정으로 
<title>★ mbc PD수첩(미친 아파트값의 비밀ㅡ2탄)"대통령은 답하라" &gt; 국민청원 &gt; 대한민국 청와대</title>
body: ㅡ집은 둥지다 (가장 기본적인 권리를 국가가 보장해야 한다) ㅡ울나라 국토좁고 인구밀도 높
<titl

<title>임산부석은 누굴위한 자리인가요? &gt; 국민청원 &gt; 대한민국 청와대</title>
body: 저는 임산부입니다 매일 지하철로 출퇴근이 3-40분 정도 됩니다 하지만 임산부석엔 매일 남
<title>대통령님..종북선언.비핵화보다 더 중요한건.. &gt; 국민청원 &gt; 대한민국 청와대</title>
body: 남북관계 개선도 물론 중요하지만 현시국에서 먼저 돌아보셔야할 문제는.. 지금 처한 나라 상
<title>일부다처제 합법화 해주세요 &gt; 국민청원 &gt; 대한민국 청와대</title>
body: 저출산문제 혼자 해결하겠습니다 
<title>저출산대책보다 기업의 사람존중이우선 &gt; 국민청원 &gt; 대한민국 청와대</title>
body: 입니다  왜냐면 우리나라는 영토가 좁거 인구가 많으니 과경쟁으로 인해 많은사람들이 스트레스
count: 70
<title>극악무도한 범죄에 경종을 울릴수있는 특별법을 국회에서 입법상정해주세요 &gt; 국민청원 &gt; 대한민국 청와대</title>
body: 세상이 너무나 흉흉해져가고 있습니다 아무것도모르는 천진난만한 아이들이 성폭력과 학대로 미래
<title>집값폭등의 주범 부동산 컨설팅 사업소 등록폐지 &gt; 국민청원 &gt; 대한민국 청와대</title>
body: 저출산의 주요한 원인은 청년들이 결혼해서 안정적으로 살 의욕을 가질 수 없기 때문입니다. 
<title>양육비지급법 &gt; 국민청원 &gt; 대한민국 청와대</title>
body: 양육비를 지급하지않는 이혼한 배우자들 때문에 고통받는 한부모 양육가정을 도와주세요.. 이혼
<title>나라꼴 개판, 법 개정이 시급합니다. &gt; 국민청원 &gt; 대한민국 청와대</title>
body: 하루걸러 매번 성범죄와 각종 강력범죄를 뉴스로 접하니 멀쩡한 사람도 정신병 오게 생겼습니다
<title>근로시간 단축의 정착화를 위해서 &gt; 국민청원 &gt; 대한민국 청와대</title>
body: 안녕하세요 정권이 바뀌며 여기저기 많

<title>난임정책 나이제한 횟수제한 없애주시고, 지원금 다시 검토해주세요!! &gt; 국민청원 &gt; 대한민국 청와대</title>
body: 저출산. 인구절벽.  요즘 젊은이들이 아이를 낳고 싶지 않아서 저출산 문제가 생긴다는 뉴스
<title>여자남자 성차별 없이 군대가게 해주세요 &gt; 국민청원 &gt; 대한민국 청와대</title>
body: 남자여자 남자는 군대가고 여자는 군대않간다는건 평등하지않습니다 물론 여자분들이 아이를낳죠 
<title>사립유치원을 초등학교 병설유치원(공교육화)으로 운영검토 하는 것은 어떨지요. &gt; 국민청원 &gt; 대한민국 청와대</title>
body: 요즘 사립유치원의 비리실태를 보면서 아이를 키우는 아버지로 매우 마음이 아프고 답답하네요.
<title>한부모가정지원혜택.. &gt; 국민청원 &gt; 대한민국 청와대</title>
body: 안녕하세요.  한부모가정이 되면서 아이를 혼자 양육하고 있습니다.  양육비는 대출이있는집받
<title>출산장려금정책에 대해서 &gt; 국민청원 &gt; 대한민국 청와대</title>
body: 정읍에 살고 있는 정읍시민입니다 다름이 아니라 저출산 대책으로 정부에서 지원하는 정책으로 
<title>유아 교육 환경 개선 국공립 어린이집, 유치원 증설 &gt; 국민청원 &gt; 대한민국 청와대</title>
body: 매년 아니 분기별 자료나 뉴스를 보면 저출산이다고 하지만 정작 국가에서는 이렇다할 대책이나
count: 140
<title>문재인 정부 탈원전 정책 반대에 대한 대응 &gt; 국민청원 &gt; 대한민국 청와대</title>
body: 탈원전 정책의 필요성과 반대에 대한 대응 현 정부의 탈원전 정책에 대해 많은 사람이 정책이
<title>유치원을 없애주세요 &gt; 국민청원 &gt; 대한민국 청와대</title>
body: 학대와 비리와 불법이 난무하는 어린이집과 유치원을 없애 주세요. 저출산 시대에 초,중고는 
<title>국민임대 소득 산출 시 반영 해주세요. 

<title>출산률을 높이는 단기계획 난임가정 지원을 늘려주세요 &gt; 국민청원 &gt; 대한민국 청와대</title>
body: 10월 10일 임산부의 날을 맞아 정부의 저출산 계획에 의견을 제시해봅니다 저는 첫째아이를
<title>▒ 헬조선의 주범 : 서울+대기업+적폐 &gt; 국민청원 &gt; 대한민국 청와대</title>
body: 공무원들은 제대로 일을 하고 있는것인가 십상시는 얼마나 존재하는가 나라가 비정상인줄 알면서
<title>아기엄마들 대중교통 편한세상? &gt; 국민청원 &gt; 대한민국 청와대</title>
body: 내년부터 카시트없으면 대중교통이용하는게 힘들어진다네요 그럼 아기가 갑자기 아프면 어떻게해야
<title>저출산 관련 대책으로 난임 지원 횟수 제한을 없애주세요. &gt; 국민청원 &gt; 대한민국 청와대</title>
body: 많은 난임 부부들이 아이를 갖고 싶어서 고통받고 있습니다. 결국 시험관아기 시술을 선택하지
<title>부동산정책 &gt; 국민청원 &gt; 대한민국 청와대</title>
body: 부익부 빈익빈. . . 우리나라가 아무리 민주공화국이라지만. . .자기돈 자기가 맘대로 쓸
<title>국제결혼 허용하자 &gt; 국민청원 &gt; 대한민국 청와대</title>
body: 국제결혼 허용해야됩니다 저출산으로인해    아기도부족합니다 그리고 결혼할 미혼남녀가 부족해
<title>불임부부에 시험관아기 &gt; 국민청원 &gt; 대한민국 청와대</title>
body: 저출산이라고 우리나라는 많은 고민도하고 하지만 정작 불임부부들에 시험관애기에 대해서는 아무
<title>동성 결혼을 하는 것을 허락 해주세요 &gt; 국민청원 &gt; 대한민국 청와대</title>
body: 안녕하세요 저는 울산에 사는 초등학교 6학년입니다. 여러 동성애 글을 보면서 생각을 해보았
count: 210
<title>여군 징병제를 찬성합니다. &gt; 국민청원 &gt; 대한민국 청와대</title>
body: 현재 한국 군대는 과도

<title>난임지원정책-첫아이라도 횟수나이제한 없애주세요 &gt; 국민청원 &gt; 대한민국 청와대</title>
body: 부부 7쌍중 1집이 난임이라고 하는데, 통계보다 실제는 더 심하겠죠.  일부러 낳지 않는 
count: 270
<title>부동산폭등!! &gt; 국민청원 &gt; 대한민국 청와대</title>
body: 10년간 지지했던 문재인 저희가족모두 지지철회합니다.  일자리, 부동산, 저출산, 최저임금
<title>태아보험에 관한 정책좀 도와주세요 제발 부탁드립니다. &gt; 국민청원 &gt; 대한민국 청와대</title>
body: 안녕하세요 올해 37세  예비아빠입니다. 결혼후 5년간 아이가 생기지 않아 1년꼬박 병원다
<title>빛좋은 개살구, 청년우대청약통장의 가입조건과 내용을 개선해주세요. &gt; 국민청원 &gt; 대한민국 청와대</title>
body: 딸아이가 서울에 살고 있어서 주거문제 때문에 걱정이 많은 지방에 사는 63세 아버지입니다.
<title>주택청약 그들만의 잔치 이제는 바뀌어야 합니다. &gt; 국민청원 &gt; 대한민국 청와대</title>
body: 주택청약의 정의와 방향성을 잡지 못하는 정책이 문제다. 주택청약을 하려는 사람들의 목적은 
<title>저출산 국가라면서 지원은 적고 병원비는 많으니 둘째계획은 포기 &gt; 국민청원 &gt; 대한민국 청와대</title>
body: 저출산 국가기에 출산장려 지원이 많은 줄 알았습니다. 바우처카드에 50만원 지원 50만원이
<title>문재인대통령님 경제현안 국민토론 해주세요 &gt; 국민청원 &gt; 대한민국 청와대</title>
body: 지방경제 말이 아닙니다.  서민경제도 죽어갑니다.  군산 울산 창원 구미 대구 시내 둘러보
<title>출산률에대한 100% 다른사고를가지자 &gt; 국민청원 &gt; 대한민국 청와대</title>
body: 티비에 대학교수란사람이나와  저출산때문에 미래가 없다고하고,,, 저출산에 에산을 수십조 갖
<title>과감하고 획기

<title>부동산 &gt; 국민청원 &gt; 대한민국 청와대</title>
body: 대한민국 나라가 땅이작아서 아파트만 짓고 있는건가요 아파트가격이너무 비싸서 일반서민들은 쳐
<title>외부 보다 내부를 먼저 살피시길 바라며.... &gt; 국민청원 &gt; 대한민국 청와대</title>
body: 옛날 아버님들이 밖안 일을 더 중요시 하고 집안일은 어머니께서 맡아 하셨지요 대한 민국이 
<title>아동수당과 육아수당 둘째이상은 2배로 지급해주세요 &gt; 국민청원 &gt; 대한민국 청와대</title>
body: 우리나라 출산율이 계속 떨어지고있습니다. 저출산을 해결하기위해서는 첫째아와 둘째아 이상에 
<title>저출산이 꼭 나쁜건가요? &gt; 국민청원 &gt; 대한민국 청와대</title>
body: 얘를 낳자고 경쟁적으로 돈을 지원하자고 난리인데. 사회시스템이 안바뀌면 불가합니다. 돈 몇
count: 340
<title>호적제도 부활과 아내는 남편의 성을 갖는  제도 만들자. &gt; 국민청원 &gt; 대한민국 청와대</title>
body: 요새 저출산이 국가 미래의 큰 문제다. 노무현 정부때 호적제도  위헌으로 폐지 된걸로 생각
<title>금리인상해주세요 &gt; 국민청원 &gt; 대한민국 청와대</title>
body: 대부분 빚을내어 집을 산 사람들이 대부분입니다. (시중에 유동자금만 1100조원이 넘습니다
<title>베트남같은데서 코피노데려오셈 &gt; 국민청원 &gt; 대한민국 청와대</title>
body: 베트남같은 나라에 코피노가 2만명정도?라는데 정작 한국은 저출산~ㅋ  코피노 애들 데려오면
<title>서울집값잡고 청년세대에게 희망을 줍시다 &gt; 국민청원 &gt; 대한민국 청와대</title>
body: 서울집값 폭등을보고 소위 전문가들이 서울은 집이 부족하고 더 지을땅이 없기 때문이며 양도세
<title>저출산장려지원정책 출산장려지원연금공단 설립 &gt; 국민청원 &gt; 대한민국 청와대</title>
body: 출

<title>무출산이 좋습니다 &gt; 국민청원 &gt; 대한민국 청와대</title>
body: 저출산 고령화 심하죠? 무출산 되고 노인들이 사라지면 어떻게 되는 걸까요? 현재 태내에 있
<title>베네수엘라, 콜롬비아 여성을 난민으로 수용해주세요. &gt; 국민청원 &gt; 대한민국 청와대</title>
body: https://m.news.naver.com/read.nhn?mode=LSD&mid=sec
<title>복지혜택에 대해서 청원드립니다. &gt; 국민청원 &gt; 대한민국 청와대</title>
body: 안녕하세요. 한부모가정이 되면서 아이를 혼자 양육하고 있습니다. 양육비는 대출이있는집받고 
<title>대통령님 복지가 제대로 쓰여야죠 &gt; 국민청원 &gt; 대한민국 청와대</title>
body: 아니.... 저출산대책에 10년동안 수십조를 쓰고도 출산율이 더 떠어지면 뭔가 획기적으로 
<title>이글 한번만 읽어주십시요..제발요.. 할말있습니다.. &gt; 국민청원 &gt; 대한민국 청와대</title>
body: 지방에서 자녀 3명을 키우고 있는 아빠입니다.  저희부부는 연소득이 4천 약간 넘습니다. 
<title>서울집값 되돌려야 문재인정권은 성공하고  정권연장 될것이다 &gt; 국민청원 &gt; 대한민국 청와대</title>
body: 오늘 방북하여 남북정상회담에 임하는 문재인대통령과 수행원 여러분의 건승을 빌며 더 이상 한
<title>북한일자리늘리기 총력 &gt; 국민청원 &gt; 대한민국 청와대</title>
body: 도대채 뭐 하는짓이여 일본 가가와(香川)현의 주부 사카구치 미와(坂口美和·가명·40)는 일
count: 410
<title>여성가족부는 대한민국  남성들의 목소리를  들어라 &gt; 국민청원 &gt; 대한민국 청와대</title>
body: 대한민국의 남성들이 억압받고 있습니다.고통받고 있습니다 여성들의 남성을 무시하는 기고만장한
<title>다자녀특별공급 &gt; 국민청원 &gt; 대한민국 청와대</tit

<title>결혼예정자 임대 아파트 10만호 공급(신혼부부타운) &gt; 국민청원 &gt; 대한민국 청와대</title>
body: 주택 정책이 온통 가격 잡는데 촛점이 잡혔는데 대규모 공급정책을 할거면 저출산 문제와 더불
<title>여기가 정말 대한민국 인가요? &gt; 국민청원 &gt; 대한민국 청와대</title>
body: 저는 한국에서 태어난 대한민국 사람입니다. 지금껏 나라에 그렇게 많은일들이 있었어도 화가나
<title>1주택자는 대체 왜건드는겁니까 예??? &gt; 국민청원 &gt; 대한민국 청와대</title>
body: 공시지가니까 대부분의 국민들은 영향이 없다 하는데 국토부 장관께서 계속 말씀하시던 공시지가
<title>국민여러분의 동의가 필요합니다!!! &gt; 국민청원 &gt; 대한민국 청와대</title>
body: 안녕하세요 . 저는 26살 67일된 딸아이를 키우는 미혼모입니다 대통령님께서 보내주신 추석
<title>남북 자유로이 왔다 갔다 하게 할 수 있다 &gt; 국민청원 &gt; 대한민국 청와대</title>
body: 눈물로 헤어지는 이산가족들을 大統領은 보지 못하였는가? 일년에 일이백명이 만나는데 언제 5
<title>답을 찾지 못하는 무능한 청와대 &gt; 국민청원 &gt; 대한민국 청와대</title>
body: 그동안 계속 말해왔다. 북한문제, 저출산문제, 청년일자리 문제 부동산문제등 정부출범 일 년
<title>여성징병은 이제 선택이아니라 필수입니다 &gt; 국민청원 &gt; 대한민국 청와대</title>
body: 이미 저출산국가를 넘어 초고령화,신생아수는 제로시대에 가까워졌습니다. 이런사태에서 정부는 
<title>저출산 문제와 최근 부동산 &gt; 국민청원 &gt; 대한민국 청와대</title>
body: “1주택자 종부세 부과기준 공시가격 9억→6억원 이상으로 확대 '저울질'  세율 정부안 대
<title>결혼예정자 및 신혼부부에게 59m2의 주택을 20~30년 상황조건으로 정부(LH)가 의무적으로 공급하자!

<title>어머니랑 티비보면서 저출산애기를 나눴습니다. &gt; 국민청원 &gt; 대한민국 청와대</title>
body: 43세장애를 갖고 있는  보령 남자입니다.  대통령님 안녕하세요.  저는 저출산에 대해 엄
count: 540
<title>지금도 열심히 일하고있는 직장인입니다. &gt; 국민청원 &gt; 대한민국 청와대</title>
body: 안녕하세요. 저는 지금도 열심히 일하고있는 직장인입니다. 지금 연봉이 3600정도 되는데 
<title>서울집값 시골집값 &gt; 국민청원 &gt; 대한민국 청와대</title>
body: 집값잡는다 못잡는다..말만많고...시간만가고....대책나오는말들어보면 그럴싸하게맞추기만하고
<title>모병제로 바꾸자 &gt; 국민청원 &gt; 대한민국 청와대</title>
body: 1) 지금은 정예강군 시대 더 이상 인해전술로 전투를 하는 시대가 아니라는 게 모병제를 찬
<title>정권이 국민에게 신뢰 받을수 있는 정책 &gt; 국민청원 &gt; 대한민국 청와대</title>
body: <정권이 국민에게 신뢰 받을 수 있는  정책> 1.경제를 살릴 수 있을 때. 2.부동산을 
<title>광주광역시 투기지역지정하라  정부는 저출산만든것 책임져야합니다 &gt; 국민청원 &gt; 대한민국 청와대</title>
body: 부동산가격이 높은데 누가 아기낳아 기르겠습니까 
<title>부동산 투기를 잡고 정말에 빠진 청,장 년층에게 희망을.. &gt; 국민청원 &gt; 대한민국 청와대</title>
body: 저는 대전유성에 거주하는 사람으로서 그동안 부동산 정책에 대하여, 많은 생각을 하고 해결할
<title>용산기지 임대주택 공급 &gt; 국민청원 &gt; 대한민국 청와대</title>
body: 현재의 정부의 부동산정책은 무능 그차체라 생각합니다. 보유세를 통한 규제라더니 찔금 무슨 


HTTPError: HTTP Error 502: Bad Gateway

In [20]:
# 멈춘 부분부터 이어서 크롤링하기

# add_body = []
count = 1246

for link in df.Link[1246:]:
    url = link
    html = urlopen(url)
    soup = BeautifulSoup(html, 'lxml')
    print(soup.title)
    
    body = soup.find('div', 'View_write').get_text()
    body = body.replace('\r', '').replace('\n', '').replace('\t', '')
    print('body:', body[:50])
    add_body.append(body)
    
    count += 1
    sleep(1.3)
    if count % 10 == 0:
        print('count:', count) 

<title>신생아를 거부하는  대학병원 &gt; 국민청원 &gt; 대한민국 청와대</title>
body: 안녕하십니까  정부정책에대해  의문이들어....이렇게  글을올려봅니다 제글을보시고 들어주실
<title>저출산대책정책제안 &gt; 국민청원 &gt; 대한민국 청와대</title>
body: 일이 터지면 아무대나 세금으로메꿀려하지말고  근본적인 원인을 찾읍시다 1 ㆍ 우리아이들을 
<title>정책제안 &gt; 국민청원 &gt; 대한민국 청와대</title>
body: 국정운영에 수고가 많으십니다 저는 경남통영시에 살고있는 50대후반의 벤처농업 6차산업을 경
<title>저출산/고령화대책 (경알못이 제안합니다) &gt; 국민청원 &gt; 대한민국 청와대</title>
body: 아무래도 대한민국 나라 돌아가는 모양이 영 아니여서 지극히 주관적인 견해로 글을 쓴다. 앞
count: 1250
<title>저 출산 문제 대책 정말 제발 간곡히 부탁 드립니다. &gt; 국민청원 &gt; 대한민국 청와대</title>
body: 글쎄요..?허~! 이 저출산 문제라 카는게, 아무래도 교육비도 많이 들고 대학 등록금도 많
<title>예비군 훈련기간을 늘리고, 더 철저하게 운영했으면 합니다. &gt; 국민청원 &gt; 대한민국 청와대</title>
body: 요즘 군복무 단축이 국방력을 저하시킨다는 이야기가 많이 나오고 있습니다. 물론 그럴 수 있
<title>어린이집 cctv실시간 볼수있게 해주세요 &gt; 국민청원 &gt; 대한민국 청와대</title>
body: 아파트에 아이들이 다니는길 , 놀이터도 cctv도 요즘 다 tv로 봅니다. 어린이집 선생님
<title>누진세 폐지해주세요. &gt; 국민청원 &gt; 대한민국 청와대</title>
body: 안녕하세요 저는 임신 9개월차 만삭임산부입니다. 당장 다음 달에 출산하는데  이 무서운 폭
<title>남성 육아휴지 의무 적용제도 &gt; 국민청원 &gt; 대한민국 청와대</title>
body: 저출산 대책은

<title>제발 누진세의 고통에서 벗어나게 서민정책펼쳐주세요. &gt; 국민청원 &gt; 대한민국 청와대</title>
body: 해마다 여름이면 반복되는 전기세누진제로 인한 고통. 집에서 제일비싸게산 가전제품임에도 여름
<title>다자녀 가정에 국가유공자 대우를 해줍시다. &gt; 국민청원 &gt; 대한민국 청와대</title>
body: 최근 저출산 문제가 심각합니다. 정부에서도 여러 장려 정책을 시행하고 계획하고는 있지만, 
<title>신혼희망 &gt; 국민청원 &gt; 대한민국 청와대</title>
body: 신혼희망타운 정말 취지는 좋습니다. 저출산의 원인으로 비혼과 만혼이 큰 비중을 차지하는 거
<title>국가공무원 육아시간 확대에 따른 장애아를 가진 공무원들의 아픔 고려 및 비공무원들의 육아시간 확대 청원 &gt; 국민청원 &gt; 대한민국 청와대</title>
body: 7월 2일 부터 국가공무원 복무규정 개정을 통해 공무원의 육아시간이 확대되었습니다.  하지
<title>공공분양의 일반분양 물량 확대 및 저소득노인층의 특별물량 배정 청원 &gt; 국민청원 &gt; 대한민국 청와대</title>
body: 공공분양의 일반분양 물량 확대 및 저소득노인층의 특별물량 배정 청원  1. 현재의 공공분양
count: 1320
<title>다자녀혜택을차별없이일륜적으로평등해주셨으면합니다 &gt; 국민청원 &gt; 대한민국 청와대</title>
body: 국가에서는 저출산이 문제다 하면서 저처럼 다자녀인 사람한테는 정작 혜택을 한번도 제대로 받
<title>문재인은 난민에 대한 입장을 확실하게 발표해라 &gt; 국민청원 &gt; 대한민국 청와대</title>
body: 더 이상 대통령으로 생각하지않기에 대통령이라 하기도 싫다. 아베나 트럼프같은 자국민보호정책
<title>아이들의 안전을 지켜주십시요 . &gt; 국민청원 &gt; 대한민국 청와대</title>
body: 안녕하십니까. 저는 4살.3살 두아이를 키우는 엄마입니다. 저희 아이들은 현재 어린이

<title>국토교통부 정책 문제 &gt; 국민청원 &gt; 대한민국 청와대</title>
body: 저출산 대책으로 신혼부부를 위한 대책이 만들어지고 있는 과정에 있습니다. 근본적 원인이 주
<title>비리를 없애자 &gt; 국민청원 &gt; 대한민국 청와대</title>
body: 이렇게 말도안되는곳으로 모르고 썼습니다. 이런 말 하시는분들 어떻게 믿을 수 있나요? 이참
<title>저출산대책및방안 &gt; 국민청원 &gt; 대한민국 청와대</title>
body: 저출산에 대해서 저의 생각을 적어볼까합니다 전 셋아이를 키우고있는 직장인아빠입니다 지금현재
count: 1380
<title>아이 키우기 좋은 나라로 만들어 주세요 &gt; 국민청원 &gt; 대한민국 청와대</title>
body: 존경하는 문재인 대통령님  최근 한 아이는 어린이집 통학버스에서 질식사를,  11개월 아이
<title>우리나라 아이들을 구해주세요 &gt; 국민청원 &gt; 대한민국 청와대</title>
body: 며칠째 어른들의 부주의와 무책임으로 인해 아이들이 사망하는 안타까운 소식을 듣고 있습니다.
<title>다자녀가족의 어려움 &gt; 국민청원 &gt; 대한민국 청와대</title>
body: 현재 초5(이란성 쌍둥이)와 큰딸(중1),세자매를 두고 있는 직장맘입니다. 하루하루 삶을 
<title>어린이집 사고 처벌 강화해주세요 &gt; 국민청원 &gt; 대한민국 청와대</title>
body: 안녕하세요 곧 21개월 되어가는 아이 엄마 입니다 요즘 어린이집에서 아이들이 억울하게 죽어
<title>세월호, 가습기살균제, 저출산 대책만큼, 어린이집 안전 대책 마련 중요하고 시급합니다. &gt; 국민청원 &gt; 대한민국 청와대</title>
body: 세월호 만큼, 가습기살균제 만큼, 저출산 대책만큼, 어린이집 안전 대책 마련 중요하고 시급
<title>보육지원제도 &gt; 국민청원 &gt; 대한민국 청와대</title>
body: 저출산에 인구감소가 심각한 상황에 어린이집

<title>조경태 의원님의 난민법폐지, 무비자폐지에 정치인들은 동참해서 통과시켜주세요 &gt; 국민청원 &gt; 대한민국 청와대</title>
body: http://www.newsis.com/view/?id=NISX20180712_000036
<title>군대 여자도 가야하나요 ? &gt; 국민청원 &gt; 대한민국 청와대</title>
body: 여자도 군대를 가야하나요 ? 이게 요즘 페이스북에서도 난리이고요 , 친구 사이에서도 난리입
<title>저출산 아이디어 &gt; 국민청원 &gt; 대한민국 청와대</title>
body: 아동수당을 3째 부터 파격적으로 주면 저출산 개선에 효과가 클것이으로 생각됩니다. 2째까지
<title>선취업 후진학을 독려합시다. &gt; 국민청원 &gt; 대한민국 청와대</title>
body: 정부나 언론에서 많이 걱정하고 있는 취업룰, 저출산 문제 근본적인 원인부터 찾아야 됩니다 
<title>반려동물을 키우는 사람들에게 책임을 주세요 &gt; 국민청원 &gt; 대한민국 청와대</title>
body: 안녕하십니까 저는 20대 청년 입니다. 요즘 귀엽고 재롱도 많이 부릴줄 아는 반려동물을 키
<title>저출산 해결을 위해 사용된 126조 사용여부 조사 &gt; 국민청원 &gt; 대한민국 청와대</title>
body: 요즘 저출산이 너무 심각하여서 출산율이 1.0이하로 내려간다는 이야기가 있습니다 저출산 해
<title>자국민 역차별금지법을 만들어주세요 &gt; 국민청원 &gt; 대한민국 청와대</title>
body: 이 나라는 뭔데 왜 다문화가정?에게만 혜택이 있나요? 국민은 사람 아닌가요?인권단체는 왜 
count: 1450
<title>워마드 남아살해 유언비어 철저히 조사하라 &gt; 국민청원 &gt; 대한민국 청와대</title>
body: 성체 훼손 게시글로 신성모독 논란까지 일으켰던 극단적 여성주의(페미니즘) 커뮤니티 '워마드
<title>예비군 기간 감축및 처우개선 &gt; 국민청원 &gt; 대한민국 청

count: 1510
<title>초,중등교사의 1학기당 1호봉이라는 학력호봉 인정은 꼭 필요한가요? &gt; 국민청원 &gt; 대한민국 청와대</title>
body: 교사들이 임용이 되면 학력호봉을 인정받아 9호봉부터 시작한다고 합니다. 4년동안 학교에서 
<title>외노자 영주권주는 외국인노동자 영구체류제도 폐지해주세요 &gt; 국민청원 &gt; 대한민국 청와대</title>
body: http://news.hankyung.com/article/2017071945721  3D
<title>유아, 어린이, 청소년들을 범죄로 부터 보호하는 특별 기구 창설 청원 &gt; 국민청원 &gt; 대한민국 청와대</title>
body: 어린이 범죄가 날로 증가하는 경향에 부모로써 걱정이 이만 저만이 아닙니다. 어린이 범죄 예
<title>저출산/고령화 문제 청원합니다. &gt; 국민청원 &gt; 대한민국 청와대</title>
body: 안녕하세요?  34살에 아직도 결혼을 못하고 서울에서 회사 직장생활을 하고 있는 일하는 청
<title>대한민국의 모든 여학생들에게 자녀안심귀가 서비스를 실시하라. &gt; 국민청원 &gt; 대한민국 청와대</title>
body: 이제는 사람을 믿지 못하는 시대, 공동체가 파괴되어져 가고 있고, 인권을 악용하는 시대에 
<title>안정된 서민주거공간 확보 및 경력단절여성 취업확대 외 &gt; 국민청원 &gt; 대한민국 청와대</title>
body: 안녕하세요~  저는 울산에서 태어나 학업, 취업, 결혼, 육아까지 쭉 해오고 있는 울산토박
<title>문재인 대통령 정부에게 저출산/고령화 문제 청원합니다. &gt; 국민청원 &gt; 대한민국 청와대</title>
body: 안녕하세요?  34살에 아직도 결혼을 못하고 서울에서 회사 직장생활을 하고 있는 일하는 청
<title>2,500원의 TV수신료 중KBS에 90%,  EBS에 3%인 70원이 할당되는 불합리한 배분을 바꿔야 합니다! TV수신료산정위원회(가칭) 설치와 EBS수신료

...

<title>금수저, 흙수저 가 없는 모두 다 잘 사는 세상 만들어주세요. &gt; 국민청원 &gt; 대한민국 청와대</title>
body: 금수저, 흙수저 가 없는 세상 만들어주세요. 존경하는 문재인 대통령님. 가난한 사람들은 결
<title>대학 커터제 &gt; 국민청원 &gt; 대한민국 청와대</title>
body: 전에 청원 했는데 반응이 없어 다시 청원 합니다. 우리나라는 현재 고3학생 나이 때의 80
<title>1가구 2주택부터 부동산 양도세 100% 과세 지금 당장 시행 해주세요!! &gt; 국민청원 &gt; 대한민국 청와대</title>
body: 부동산 투기는 투자가 아닙니다. 범죄 행위입니다  1가구 2주택부터 부동산 매도시 양도소득
count: 3050
<title>아이들이 고통받아야 하는 사회 &gt; 국민청원 &gt; 대한민국 청와대</title>
body: 요즘 tv만 틀면 아동학대로 인한 사고가 빈번합니다 저도 한 아이의 부모로서 경악을 금치 
<title>가상화폐관련 국민ㆍ 정부관계자 분들 한번만 읽어주세요 &gt; 국민청원 &gt; 대한민국 청와대</title>
body: 이글은 제가 직접 쓴글이 아닙니다 청원 20만 넘자고 쓰는글도 아닙니다 이글이 누구의 글인
<title>워킹맘을 위한 시간선택제(10시~4시)를 보편화 해 주세요! &gt; 국민청원 &gt; 대한민국 청와대</title>
body: 안녕하세요? 20개월 아들이 있는 워킹맘입니다.  실제 복직 후 생활을 해보니, 워킹맘에게
<title>저출산해결책 가져왔습니다! &gt; 국민청원 &gt; 대한민국 청와대</title>
body: 안녕하세요  16개월 아이를 집에서 기르고 있고 아이 낳기전까진특수기간제교사였던 32살 대
<title>경제가 위험한 것 같네요. 2021년도정도에 폭발할 것 같네요-_- &gt; 국민청원 &gt; 대한민국 청와대</title>
body: 경제가 위험한 것 같네요.  직장있는 사람은 지금 내수경제가 어려운 것을 모른다네요.  문
<t

<title>북한위협 막을 길 있다 &gt; 국민청원 &gt; 대한민국 청와대</title>
body: 5개월 전부터 말해왔다 3개월 내 김정은이 7차 핵실험을 하던가 미국까지 날아가는 미사일을
<title>다자녀가정 지원대책 확대 &gt; 국민청원 &gt; 대한민국 청와대</title>
body: 정부에서는 저출산으로 인한 여러가지 정책들을 펼치고 있지만 피부에 와 닫는것이 없습니다 저
<title>IT업종 종사자도 정규직으로 전환해주세요~~!!! &gt; 국민청원 &gt; 대한민국 청와대</title>
body: 안녕하세요...  대통령님께 꼭 말씀드리고 싶었습니다.  특히 IT업종(서버운영관리, 구내
<title>헌법개정을 통한 "남여 공평한 국방의 의무"를 갖도록 하자.(대체복무제를 활성하여 모든 육아를 국가가 책임지도록) &gt; 국민청원 &gt; 대한민국 청와대</title>
body: 현대전은 국가 총력전이기 때문에 넓은 의미의 국방의 의무에는 병역의 의무뿐만 아니라 방공·
<title>저출산 대책,  발상의 전환이 필요합니다 &gt; 국민청원 &gt; 대한민국 청와대</title>
body: http://v.media.daum.net/v/20180124120011230?rcmd=r
<title>저출산 &gt; 국민청원 &gt; 대한민국 청와대</title>
body: 그리고. 또. 하나. 말씀드리게요. 다문화정책이나이민을. 문재인 대통령께서는 반대하시는것같
count: 3120
<title>저출산 &gt; 국민청원 &gt; 대한민국 청와대</title>
body: 이민 허용하라는 말씀을드리고싶고.  이민허용을. 해야지. 노동력 부족과. 생산인구 감소도 
<title>저출산 &gt; 국민청원 &gt; 대한민국 청와대</title>
body: 이민을반대하는건. 좋지않을것 같습니다  그리고. 이민을받아들이는것도.  고려해주시길. 바랍
<title>저출산 &gt; 국민청원 &gt; 대한민국 청와대</title>
body: 우수외국인력이나. 이민도받아들여

<title>산부인과 비용 부담 &gt; 국민청원 &gt; 대한민국 청와대</title>
body: 안녕하십니까 저는 19살 고등학교를 재학중인 여성입니다.  제가 청원을 쓰게된 이유는 산부
<title>(장문글5가지 청원) 지르기만 하는 정책 소용없습니다. &gt; 국민청원 &gt; 대한민국 청와대</title>
body: 1. 최저시급 인상 정부는 최저시급을 인상했을때 발생할 수 있는 문제점을 알고도   추진했
<title>20만 만들어서 청와대 답변좀 들어봅시다.. 미세먼지.. 우리 아이들 다 죽일 작정입니까? &gt; 국민청원 &gt; 대한민국 청와대</title>
body: 안녕하세요.. 정말!! 안녕하십니까? 오늘 출근 잘 하셨나요? 느끼는거 없었나요? 오늘 무
<title>북한위협 막을 길 있다 &gt; 국민청원 &gt; 대한민국 청와대</title>
body: 4개월 전부터 말해왔다 3개월 내 김정은이 7차 핵실험을 하던가 미국까지 날아가는 미사일을
<title>공교육 강화로 출산율과 학생행복률을 높여주세요 &gt; 국민청원 &gt; 대한민국 청와대</title>
body: 안녕하십니까 서울에 사는 한 시민입니다. 최근에 뉴스에서 저출산으로 서울의 한 사립 초등학
<title>공교육 강화로 출산율과 학생행복률을 높여주세요 &gt; 국민청원 &gt; 대한민국 청와대</title>
body: 안녕하십니까 서울에 사는 한 시민입니다. 최근에 뉴스에서 저출산으로 서울의 한 사립 초등학
count: 3190
<title>재산신고에 암호화폐는 왜 빼는겁니까? &gt; 국민청원 &gt; 대한민국 청와대</title>
body: 우리나라에선 아주 나중엔 암호화폐로 방향을 잡을지는 모르나  지금 현 상황은 정부는 국민과
<title>아기 양육수당 &gt; 국민청원 &gt; 대한민국 청와대</title>
body: 이제 아기를 낳은지 12일 된 30대 초반 주부입니다. 임신 5개월차에 일 특성상 쉬게 되
<title>  1가구 2주택자부터  주택처분시 양도소득세 

count: 3250
<title>입학생이 줄어든 초등학교 시설을 발달장애 관련 특수학교와 공유할 수 있었으면 합니다. &gt; 국민청원 &gt; 대한민국 청와대</title>
body: 안녕하세요.. 저는 발당장애 2급 우리나이로 9세 이번에 초등학교 2학년에 올라가야할 나이
<title>강남부동산 고위공직자들이 솔선수범안하니 .. 이모양이죠.. &gt; 국민청원 &gt; 대한민국 청와대</title>
body: 제가 계속 주장했습니다.  주머니 만지작거리지말고 흉내만내지말라고요... 버티기 일관...
<title>다자녀혜택이요.. &gt; 국민청원 &gt; 대한민국 청와대</title>
body: 아이 넷엄마 입니다. 남편월급 보너스포함 280평균입니다 그런데도  나라에서하는 산후조리 
<title>매일 그렇게 규제만하면서 서민의 숨통까지 끊을 작정입니까? 가상화폐 폐지가 아니라 오히려 정부가 사회안전망 시스템을 만들고 새로운 방향을 찾아줄 수는 없나요? &gt; 국민청원 &gt; 대한민국 청와대</title>
body: 뉴스만보면 온통 '규제' '규제' '규제' 뿐입니다.  공부하면 성공할수있다 1등하면 성공
<title>대가족 제도를 정부차원에서 장려-적극적으로 지원 해주세요=&gt;저출산 문제 해결방안을 제안-청원 합니다. &gt; 국민청원 &gt; 대한민국 청와대</title>
body: 대가족 제도를 정부차원에서 장려-적극적으로 지원 해주세요=>저출산 문제 해결방안을 제안-청
<title>임신하고싶습니다. &gt; 국민청원 &gt; 대한민국 청와대</title>
body: 안녕하세요.지방에살고있는 30대새댁입니다. 혹시 맞춤법이나 글에 두서가 없더라도 이해해주세
<title>학교 돌봄교실을 전학년으로  확대해주세요. &gt; 국민청원 &gt; 대한민국 청와대</title>
body: 초등 돌봄교실을 저학년(1,2학년) 에서 전학년으로 확대 요청 드립니다.  교육부, 여성가
<title>수원중앙기독중학교 학급증설 부결사유 &gt; 국민청원 &gt; 

<title>미성년자 성범죄 처벌 수위 강화 &gt; 국민청원 &gt; 대한민국 청와대</title>
body: 미성년자 성범 처벌 수위 강화를 원합니다. 최소 무기징역이나 또는 사형으로 법을 만들어주세
<title>아동성범죄자들의 천국이 대한민국이래요 &gt; 국민청원 &gt; 대한민국 청와대</title>
body: 이제곧 딸의 아빠가 되는입장에서 사회가 너무 두렵고 무섭습니다. 무슨놈의 나라는 세금잘내고
<title>끊이지않는 아동성범죄. &gt; 국민청원 &gt; 대한민국 청와대</title>
body: http://v.media.daum.net/v/20180103205701075?f=m&rc
<title>저출산은 대한민국 미래가 없습니다.. &gt; 국민청원 &gt; 대한민국 청와대</title>
body: 우리나라 미래를 생각한다면 북한의 핵보다 더 무서운 것이 저출산 고령화라고 말씀드리겠습니다
count: 3320
<title>배곧신도시 초등학교돌봄교실 학급충원에 대하여 &gt; 국민청원 &gt; 대한민국 청와대</title>
body: 안녕하세요 저는 2017.10.8 일 배곧신도시에 입주한 맞벌이 세대입니다. 저는 1학년,
<title>아이돌봄서비스/어린이집 (보육) 재정비 및 지원 확대 &gt; 국민청원 &gt; 대한민국 청와대</title>
body: 저출산의 원인 중 가장 크게 차지하는 아이 보육과 관련하여 청원합니다. 모든 맞벌이 가정이
<title>출산 장려 정책 &gt; 국민청원 &gt; 대한민국 청와대</title>
body: 저출산 문제는 인구의 절벽을 가져오는 가장 시급한 정부시책입니다.  출산 장려정책이 여러가
<title>중소기업 재직 8개월차... 임신했다고 말씀드렸더니  회사 나가라고 합니다. &gt; 국민청원 &gt; 대한민국 청와대</title>
body: 안녕하십니까? 경기 모 중소기업 다니고  있는  임신 6주차 곧 엄마가 될 사람입니다. 어
<title>국정 최우선 순위 저출산 대책 &gt; 국민청원 &gt; 대

In [21]:
len(add_body)

3339

In [23]:
df.Link[3338]

'https://www1.president.go.kr/petitions/77810'

In [24]:
df['본문'] = add_body
df.head()

,Unnamed: 0,제목,청원시작,청원인원,Link,본문
0,0,저출산 관련 육아휴직 및 어린이집 증설,2018-10-31,3,https://www1.president.go.kr/petitions/427334,300인 미만 기업에 다니고 있는 2018년도에 출산한 아기 엄마입니다. 먼저 회사...
1,1,저출산 ↔ 어린이집 부족. 서로 너무 상충되는 관계 아닌가요?,2018-10-31,7,https://www1.president.go.kr/petitions/427316,"안녕하세요. 천안시 백석동에 사는 한 시민입니다. 천안은 (특히 백석, 불당) 전국..."
2,2,어린이집을 유지해주세요,2018-10-31,476,https://www1.president.go.kr/petitions/426902,양천구 신월동에 위치한 *** 국공립 어린이집에 6살 아이를 보내는 엄마입니다. 4...
3,3,오늘 안하면 늦습니다. 누구나 마음놓고 출산할 수 있는 제도 마련 시급합니다.,2018-10-31,2,https://www1.president.go.kr/petitions/426884,안녕하세요 문재인대통령님^^ 국민의 한사람으로서 또한 애를 키우고 있는 가장으로서 ...
4,4,가족돌봄휴가제도를 조속히 시행해야 합니다.,2018-10-31,2,https://www1.president.go.kr/petitions/426830,올해 2월에 신문들에 일제히 보도된 가족돌봄휴가제도에 대한 내용입니다. [한겨레 2...


In [37]:
df.columns

Index(['Unnamed: 0', '제목', '청원시작', '청원인원', 'Link', '본문'], dtype='object')

In [25]:
del df['Unnamed: 0']

In [26]:
df.head()

,제목,청원시작,청원인원,Link,본문
0,저출산 관련 육아휴직 및 어린이집 증설,2018-10-31,3,https://www1.president.go.kr/petitions/427334,300인 미만 기업에 다니고 있는 2018년도에 출산한 아기 엄마입니다. 먼저 회사...
1,저출산 ↔ 어린이집 부족. 서로 너무 상충되는 관계 아닌가요?,2018-10-31,7,https://www1.president.go.kr/petitions/427316,"안녕하세요. 천안시 백석동에 사는 한 시민입니다. 천안은 (특히 백석, 불당) 전국..."
2,어린이집을 유지해주세요,2018-10-31,476,https://www1.president.go.kr/petitions/426902,양천구 신월동에 위치한 *** 국공립 어린이집에 6살 아이를 보내는 엄마입니다. 4...
3,오늘 안하면 늦습니다. 누구나 마음놓고 출산할 수 있는 제도 마련 시급합니다.,2018-10-31,2,https://www1.president.go.kr/petitions/426884,안녕하세요 문재인대통령님^^ 국민의 한사람으로서 또한 애를 키우고 있는 가장으로서 ...
4,가족돌봄휴가제도를 조속히 시행해야 합니다.,2018-10-31,2,https://www1.president.go.kr/petitions/426830,올해 2월에 신문들에 일제히 보도된 가족돌봄휴가제도에 대한 내용입니다. [한겨레 2...


In [27]:
df['본문'][0]

'300인 미만 기업에 다니고 있는 2018년도에 출산한 아기 엄마입니다. 먼저 회사에서는 출산휴가 3개월과 육아휴직6개월을 주어 총 9개월을 아기를 돌보게 되었습니다. 내년4월 복직해야 하나 돌도 안된 아이를 맡길곳이 마땅치 않습니다. 어린이집 대기를 걸었으나 순위에서 밀려 대기는 계속 늘어가고 있고, 양가 부모님은 아기을 돌봐줄 상황이 되지 않습니다. 그러면 출근과 동시에 육아 도우미를 고용해야하나 그러기에는 비용이 너무 만만치 않아 퇴사까지 고려하고 있습니다. 그러기에 아기가 너무 이쁘고 형제자매를 만들어주고 싶어도 둘째는 꿈꾸고 있지 않습니다. 결론적으로 두가지를 청원합니다.  1. 작은 회사일수록 육아휴직을 의무화해 고용및 재직기간을 늘려야 합니다.  대기업은 기본적으로 1년의 육아휴직과 3개월의 출산휴가를 의무적으로 사용해 충분히 아기를 돌보고 다시 복직하여 회사를 다닐 수 있습니다. 중소기업 재직자들은 결국 퇴사를 하게되어 결국 부익부 빈익빈이 초래될수도 있습니다. 의무적으로 육아휴직을 사용할 수 있게 해주세요.  2. 어린이집에 만0세반도 늘리고 국공립 어린이집 수 자체를 늘려야 합니다.  중소기업자체가 육아휴직을 제공하기에 현실적으로 무리가 있다면 돌도 안된 만0세의 아가를 돌봐줄 시스템이 마련되어야 합니다. 만0세를 맡아주지 않는 어린이집도 많을 뿐더러 말도 못하는 아기를 맡기자니 국공립 어린이집이 아니면 불안감도 커지고 또한 맡길 수 조차 없는 많은 아기들이 대기를 하고 있습니다.  제발 이 두가지에 관심을 가지고 시정 부탁드립니다. ㅠㅠ '

In [28]:
df.to_csv('data/petition_crawling_low_birth_rate_with_body.csv', sep=',', encoding='UTF-8')

In [29]:
! dir data

 C 드라이브의 볼륨: BOOTCAMP
 볼륨 일련 번호: 5415-72D5

 C:\Users\JuhnnyKo\Dropbox\Anaconda_src\data 디렉터리

2018-11-14  오후 12:01    <DIR>          .
2018-11-14  오후 12:01    <DIR>          ..
2018-11-14  오전 01:45    <DIR>          .ipynb_checkpoints
2018-11-07  오후 02:29             7,686 chicagomag_top_50_sandwiches.csv
2018-11-07  오후 05:00            10,314 chicagomag_top_50_sandwiches_ver2.csv
2018-10-30  오전 09:32               310 company.csv
2018-11-11  오후 06:22            43,781 evernote_schedule_2018.txt
2018-11-01  오후 04:23                62 girlgroup.json
2018-10-29  오후 04:32                21 hello.txt
2018-11-01  오후 04:26               162 member.json
2018-10-30  오전 11:06                10 mydata3.txt
2018-10-30  오전 11:14                 5 mydata4.txt
2018-10-30  오전 11:31                 8 mydata5.txt
2018-11-09  오전 11:10             8,872 naver_mv_ranking.csv
2018-11-01  오후 04:41               456 note_181101.txt
2018-11-06  오전 09:23             1,140 opengov_2018_seoul_cctv_status_by_yea